## Sqlite3
The sqlite3 built-in module within Python. SQLite allows us to quickly get up and running with databases, without spinning up larger databases like MySQL or Postgres. We will be creating a database, creating a table, insert, select, update, and delete data.

In [1]:
class Employee:
    '''A sample employee class'''
    
    def __init__(self, first,last, pay ):
        self.first = first
        self.last = last
        self.pay = pay
        
    @property
    def email(self):
        return f'{self.first}.{self.last}@email.com'
    
    @property
    def full_name(self):
        return f'{self.first}, {self.last}'
    
    def __repr__(self):
        return f'{self.first}, {self.last}, {self.pay}'

In [2]:
import sqlite3

## 1: Create a connection 

- sqlite3.connect() method

In [3]:
conn = sqlite3.connect('employee.db')
#conn = sqlite3.connect(':memory:')# TO create in memory db i.e RAM Database
# Connect method creats the file if that doesn't exist

In [4]:
print(type(conn))

<class 'sqlite3.Connection'>


## 2: Create a cursor.
- This allow us to execute sql commands
- cursor method on connection object

In [6]:
c = conn.cursor()

## 3: Execute queries

- execute method on c

In [23]:
command = ''' CREATE TABLE employee(
                first TEXT,
                last  TEXT,
                pay   INTEGER)'''

c.execute(command)

## 4: Commit & Close 
 - On connection

In [28]:
conn.commit()
conn.close()

## Inserts toDB

In [29]:
#REMEMBER: CREATE A CONNECTION OBJECT AND CURSOR MENTHOD ON THAT OBJECT
conn = sqlite3.connect('employee.db')
c = conn.cursor()

In [31]:
Insert_statement = ''' INSERT INTO employee VALUES('Iqbal','Singh',200000)
                    '''

In [32]:
c.execute(Insert_statement)
conn.commit()

## Selection

In [34]:
conn = sqlite3.connect('employee.db')
c = conn.cursor()

In [40]:
Select_statement = '''SELECT * FROM employee '''

In [46]:
c.execute(Select_statement)

In [42]:
#fetchall, fetchone, fetch many
c.fetchall()

[('Iqbal', 'Singh', 200000)]

In [45]:
c.fetchone()

('Iqbal', 'Singh', 200000)

In [47]:
c.fetchmany(3)#returns N number of rows

[('Iqbal', 'Singh', 200000)]

## Passing variables

- use ? for placeholders ->tuple is used to pass values
- use :placeholder for placeholders ->Dictionary is used to pass values
  -:key and

In [48]:
emp_1 = Employee('John','Doe',80000)
emp_2 = Employee('Mary','Doe',80000)
emp_3 = Employee('Jim','Torey',80000)

In [49]:
#Method 1: Not so preffered, here tuple is used to pass values
c.execute('INSERT INTO employee values(?,?,?)',(emp_1.first ,emp_1.last ,emp_1.pay ))
conn.commit()

In [50]:
# Mthod 2: Right way
c.execute('INSERT INTO employee values(:first, :last, :pay )',{'first':emp_2.first,
                                                               'last':emp_2.last, 'pay':emp_2.pay })
conn.commit()

In [51]:
# Mthod 2: Right way
c.execute('INSERT INTO employee values(:first, :last, :pay )',{'first':emp_3.first,
                                                               'last':emp_3.last, 'pay':emp_3.pay })
conn.commit()

In [54]:
c.execute('''SELECT * FROM employee where last = ?''',('Doe',))#This comma after Doe is confusing
c.fetchall()

[('John', 'Doe', 80000), ('Mary', 'Doe', 80000)]

In [58]:
c.execute('''SELECT * FROM employee where last = :last ''',{'last':'Doe'})
c.fetchall()
conn.close()

[('John', 'Doe', 80000), ('Mary', 'Doe', 80000)]

# In Memory DATABASE 
- Every run creates a new db instance

In [87]:
conn = sqlite3.connect(':memory:')
c = conn.cursor()
command = ''' CREATE TABLE employee(
                first TEXT,
                last  TEXT,
                pay   INTEGER)'''
c.execute(command)
emp_1 = Employee('John','Doe',80000)
emp_2 = Employee('Mary','Doe',80000)
emp_3 = Employee('Jim','Torey',80000)
for emp in [emp_1,emp_2,emp_3]:
    c.execute('INSERT INTO employee values(:first, :last, :pay )',{'first':emp.first,
                                                               'last':emp.last, 'pay':emp.pay })
    conn.commit()
c.execute('''SELECT * FROM employee where last = ?''',('Doe',))#This comma after Doe is confusing
c.execute('''SELECT * FROM employee where last =:last''',{'last':'Torey'})#Right way
c.fetchall()

[('Jim', 'Torey', 80000)]

In [88]:
conn.close()

# Creating App like structure!

- Here we used <b>with</b> statement to pass commit typing everytime, for INSERT, UPDATE, DELETE!!

In [114]:
conn = sqlite3.connect(':memory:')
c = conn.cursor()

command = ''' CREATE TABLE employee(
                first TEXT,
                last  TEXT,
                pay   INTEGER)'''

c.execute(command)

def insert_emp(emp):
    with conn:
        c.execute('INSERT INTO employee VALUES(:first, :last, :pay)',{'first':emp.first,'last':emp.last, 
                                                                     'pay':emp.pay})
def get_emps_by_name(lastname):#Select needs no commit    
    c.execute('SELECT * FROM employee where last =:last',{'last':lastname})
    return c.fetchall()
    
def update_pay(emp, pay):#DML statement Hence, CONTEXT Manager.
    with conn:
        c.execute('''UPDATE employee SET pay =:pay
                    WHERE first =:first AND last=:last''',{'first':emp.first,'last':emp.last,'pay':pay})
def remove_emp(emp):
    with conn:
        c.execute('''DELETE FROM employee WHERE first =:first AND last =:last''', 
                 {'first':emp.first, 'last':emp.last})
    

In [115]:
emp_1 = Employee('John','Doe',80000)
emp_2 = Employee('Mary','Doe',80000)
emp_3 = Employee('Jim','Torey',80000)

In [116]:
insert_emp(emp_1)
insert_emp(emp_2)
insert_emp(emp_3)

In [117]:
emps = get_emps_by_name('Doe')
print(emps)

[('John', 'Doe', 80000), ('Mary', 'Doe', 80000)]


In [118]:
update_pay(emp_3,95000)

In [119]:
emps = get_emps_by_name('Torey')
print(emps)

[('Jim', 'Torey', 95000)]


In [122]:
remove_emp(emp_2)

In [123]:
emps = get_emps_by_name(emp_2.last)
print(emps)

[('John', 'Doe', 80000)]


In [124]:
conn.close()

In [125]:
get_emps_by_name('Torey')

ProgrammingError: Cannot operate on a closed database.